##  Welcome to the Julia Markov Decision Utility

The Julia Markov Decision Utility (MDU) is a packge written in Julia that allows users to compute and analyze how changes to decision states in a given environment impact specified outcomes, including

* the probability of ending up in key absorbing states
* the average time to absorption
* average decision entropy faced by actors



In [1]:
#
#  RUN THIS CELL FIRST
#      - NOTE, JULIA USES JIT COMPILATION, AND IT CAN RELATIVELY SLOW THE FIRST TIME IT RUNS 
#

# IMPORTANT PIECE—TELLS JULIA TO FIND THE LOCAL MODULES
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing

# Load some standard Julia packages managed by Pkg
using LinearAlgebra
using SparseArrays
using DataFrames
using Statistics
using Dates

# load local modules -- only required for base-level interactions
using DataStructures
using MarkovDecisionUtilities
using MarkovModels
using MDUSetups
using SupportFunctions

# increases the number of visible columns when printing a table in iJupyter
ENV["COLUMNS"] = 240


┌ Info: Precompiling DataStructures [top-level]
└ @ Base loading.jl:1664
┌ Info: Precompiling MarkovDecisionUtilities [top-level]
└ @ Base loading.jl:1664
[ Info: Setting project directory to '/Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities'


#  1. Interacting with data: Initialize the `MarkovDataset` structure

The MDU ingests data using an object called **`MarkovDataset`** (defined in `DataStructures.jl`). The `MarkovDataset` object requires two initialiation arguments, which are described below. Additionally, relevant data tables and associate fields (as well as field data type requirements) are detailed below.

##  `dir_dataset`
`dir_dataset` gives the directory containing the dataset to run. The base file path of the directory is assumed to be the name of the dataset in the MDU. In the MDU scripts, the directory `./ref/datasets` contains subdirectories with datasets. Each dataset must contain three files (described below):

###  Scenario Attribute Table (attribute_scenario.csv)
This file contains basic information about each scenario. It requires three fields:
    - `scenario_id` (type `Int64`): key field giving the scenario id, an integer. This index is used throughout the MDU to refer to scenarios.
    - `scenario_name` (type `String`): name of the scenario, used for tracking purposes.
    - `default_starting_state` (type `Int64`): a default starting state for the scenario to use in simulations.
        
       
###  State Attribute Table (attribute_state.csv)
`attribute_state.csv` gives information about **all possible states across all scenarios**. Different scenarios do not re-index states. They all operate on the same set of available decision states. Not every state must be used in every scenario; however, any states that are defined in a scenario must be included here, and states do not change across scenarios. This file has $2(s + 1)$ fields, where $s$ is the number of scenarios. 

This file requires the following 2 fields (independent of number of scenarios) with entried defined for each state:
    - `state` (type `Int64`): the state number, an integer index that is used to define transition matrices in `transition_matrices_by_scenario.csv`
    - `name` (type `String`): name of the state, used for tracking (can be merged back into data for reporting purposes)
        
Additionally, `attribute_state.csv` requires 2 additional fields *for each scenario defined*:
    - `time_#` (type `Int64`): the number of times to spend in this state under scenario number `#`. The number should be entered as an integer; for example, for scenario 0, `time_0` gives the time spent in the state (1 means one step; 10 means ten steps), while `time_12` would give the times in each state for scenario 12.
    - `outcome_value_#` (type `Int64` or `Float64`): value used to score ending a simulation in this state. This is used to calcualte expected outcomes and simulated outcomes. For example, values < 0 may indicate undesirable outcomes in the decision space, while positive values may indicate more desirable outcomes. These values are set by the analyst and research team and depend on the context of the decision environment and analytical goal.
        
        
###  Transition Matrices by Scenario (transition_matrices_by_scenario.csv)
`transition_matrices_by_scenario.csv` defines the transition matrices for each scenario using a sparse array schema. Transitions are added as elements in a stochastic matrix (**NOTE:** the orientation of the matrix is set in `complexity_modeling.config`. This example uses the default, a row-stochastic matrix.); i.e., to specify a transition from $1 \to 2$, ensure that there is a row where $i = 1$ and $j = 2$. The transition probability associated with scenario $\#$ is entered in a row with the name `transition_expression_scenario_#`. 

Users should keep the following in mind when entering transition matrices:
    - Transitions that aren't entered are assumed to occur with probability 0
    - A transition link must be entered as a row in this file if it exists in **any** scenario. 
    - Links that do not exist in any scenario do not have to be entered in the file.
    - In row-stochastic orientation, all entries in a row must sum to 1 (i.e., for a given scenario, all entries with the same state index `i` must sum to 1) 

`transition_matrices_by_scenario.csv` requires $2 + s$ fields. Two fields are required independent of the number of scenarios:
    - `i` (type `Int64`): the source or origin state (transition out)--must be defined in `attribute_state.csv`
    - `j` (type `Int64`): the target or end state (transition in)--must be defined in `attribute_state.csv`

Finally, the following field must be entered for each scenario defined in `attribute_scenario.csv`:
    - `transition_expression_scenario_#` (type `Float64` or `String`): Most users should only enter Float64 expressions. This field gives transition probabilities for scenario $\#$. However, it is possible to enter symbolic expressions, such as `"1 - A"` (where `A` is the symbolic variable) in this field, allowing users to evaluate a range of matrices. However, if doing so, an associated variable must be specified in the configuration file that gives a default value for `A` (for example `variable_A: 0.15`). The definition of a default symbolic variable value allows the model to run a number of scenarios without exploring over the symbolic variable. MarkovModel functions allow for users to set specific values of symbolic variables for individual runs.
        
      
##  `default_transition_scenario`

The `default_transition_scenario` gives the default scenario (integer) to run, when running a single simulation run, in the absence of a specified scenario. This scenario must be defined in `attribute_scenario.csv`.

In [2]:
dataset = MarkovDataset(
    joinpath(dir_datasets, config.dataset), 
    config.default_transition_scenario
);

####  Use Julia's `propertynames()` function to view properties and methods of the dataset structure (this can be applied to **any** of the structures discussed below)

A property/method X can be accessed using MarkovDataset.X

In [51]:
for k in sort(collect(propertynames(dataset)))
    println(k)
end

all_scenarios
attribute_scenario
attribute_state
dataset_name
default_transition_scenario
dir_dataset
field_i
field_j
field_scenario_default_starting_state
field_scenario_key
field_scenario_name
field_state_key
field_state_name
fn_attribute_scenario
fn_attribute_state
fn_transition_by_scenario
get_markov_data
get_table_field
print_valid_scenarios
required_files


We can use the `dataset.dataset_name` property to verify that we're running the dataset defined in `config`

In [36]:
println("dataset.dataset_name:\t$(dataset.dataset_name)")
println("config.dataset:\t$(config.dataset)")

dataset.dataset_name:	cuban_missile_crisis
config.dataset:	cuban_missile_crisis


###  We can verify dataset properties and input tables by using the `MarkovDataset` structure
Running the next cell will show the input table specified as `attribute_scenario.csv` for the dataset

In [38]:
dataset.attribute_scenario.table

,scenario_id,scenario_name,default_starting_state
,Int64,String,Int64
1,0,Baseline,1
2,1,Baseline - Deterrence,1
3,2,Premature Discovery,1
4,3,Premature Discovery - Escalation,1
5,4,Premature Discovery - De-Escalation,1


Running the next cell will show the input table specified as `attribute_state.csv` for the dataset, giving all of the states that are defined in this decision environment

In [39]:
dataset.attribute_state.table

,state,name,time_0,outcome_value_0,time_1,outcome_value_1,time_2,outcome_value_2,time_3,outcome_value_3,time_4,outcome_value_4
,Int64,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,Assess nuclear basing in Cuba,1,0,1,0,1,0,1,0,1,0
2,2,Move nuclear missiles to Cuba,1,0,1,0,1,0,1,0,1,0
3,3,No missiles in Cuba,1,10,1,10,1,100,1,100,1,100
4,4,Missiles remain in Cuba,1,-100,1,-100,1,-100,1,-100,1,-100
5,5,US naval blockade of Cuba,1,0,1,0,1,0,1,0,1,0
6,6,US attacks Cuba to remove missiles,1,0,1,0,1,0,1,0,1,0
7,7,Limited conflict between US and Soviet Union,1,0,1,0,1,0,1,0,1,0
8,8,All-out conflict between US and Soviet Union,1,0,1,0,1,0,1,-1000,1,-1000


####  Accessing the transition inputs

To access transition inputs, use `dataset.get_markov_data(scen)`. This function returns a tuple with two elements: 
- the first element is the attribute_state table, used to streamline calculations, and 
- the second element gives the sparse transition matrix associated with the scenario as a DataFrame

In [47]:
# get the tuple
markov_data_0 = dataset.get_markov_data(0)
# print the sparse input matrix as a dataframe
markov_data_0[2]

,i,j,expression
,Int64,Int64,Float64
1,1,2,0.5
2,1,3,0.5
3,2,3,0.5
4,2,4,0.5
5,5,5,1.0
6,6,6,1.0
7,7,7,1.0
8,8,8,1.0


##  Once a dataset is initialized, we can choose a scenario index and initialize a `MarkovModel` object

- the dataset .get_markov_data() function will validate scenarios and check if the specification is valid
- the `MarkovModel` structure performs: 
    - checks on the transition matrix;
    - transformations of the transition matrix, such as canonical form;
    - analytical solutions (where possible); and
    - simulations (at the base level).

In [4]:
# choose a valid scenario from dataset.attribute_scenario.table (from above)
scen = 4
attribute_state, df_trans = dataset.get_markov_data(scen, true);

# initialize a MarkovModel structure for scenario 4
mod_markov = MarkovModel(df_trans, attribute_state, config);


##  The MarkovModel struct includes a range of properties and methods 

These methods can leveraged against the default transition matrix (the one specified on input) or other transition matrices that are specified exogenously.

To examine some of the functionality, use ?MarkovModel (can try this below--note, the description will be long, and it may be convenient to collapse the output cell to continue navigating the notebook)

In [3]:
?MarkovModel

search: MarkovModel MarkovModels



# Information

The `MarkovModel` structure includes a range of methods for interacting with     transition matrices and random walks. The `MarkovModel` structure generally     includes the following capabilities:

```
- checks on the transition matrix;
- transformations of the transition matrix, such as canonical form;
- analytical solutions (where possible); and
- simulations (at the base level).
```

# Constructs

```
MarkovModel(
    model_specification::DataFrame
    attribute_states::AttributeTable
    configuration::Configuration
)
```

# Initialization Arguments

See `DataStructures.jl` for information on AttributeTable and Configuration     structures.

  * `model_specification`: DataFrame used to input transition matrices. The   DataFrame should include the following fields:

      * `i`
      * `j`
      * `expression`
  * `attribute_states`: AttributeTable
  * `configuration`: Configuration object

# Methods

The following methods and properties are defined in `MarkovModel`

  * `check_transition_matrix!`
  * `do_walk`
  * `find_absorption`
  * `find_next_state`
  * `get_absorbing_states`
  * `get_all_state_transitions`
  * `get_canonical`
  * `get_fundamental_matrix`
  * `get_probability_of_absorption_state`
  * `get_state_attribute`
  * `get_state_entropies`
  * `Q_default` (property: default transition matrix defined from the input   DataFrame `model_specification`)
  * `simulate_absorptions`
  * `simulate_walks`
  * `swap_inds!`
  * `swap_states!`
  * `transition_matrix_from_data_frame`

Information on these methods is provided below.

## `check_states!`

Check the validity of input states against specified attribute_states.

### Constructs

```
check_states!(
	states::Vector{Int64}
)
```

## `check_transition_matrix!`

Check the transition matrix `mat` with stochastic orientation `stochastic_orientation`     and return a valid matrix with stochastic orientation `return_orientation`.     Use `correction_thresh` to adjust invalid probability transition vectors.

### Constructs

```
check_transition_matrix!(
    mat::SparseMatrixCSC{Float64, Int64},
    correction_thresh::Float64 = configuration.transition_correction_thresh,
    stochastic_orientation::Symbol = configuration.transition_stochastic_orientation,
    return_orientation::Symbol = :row
)
```

## `format_probs_as_columns`

### Constructs

```
format_probs_as_columns(
	vec_ext_inds::Array{Tuple{Int64, Int64}, 1}
)
```

## `get_absorbing_states`

Get the absorbing states associated with Sparse matrix `mat`

### Constructs

```
get_absorbing_states(
    mat::SparseMatrixCSC{Float64,Int64}
)
```

## `get_all_state_transitions`

Get the set of all state transitions ($i \to j$) contained in `mat`

### Constructs

```
get_all_state_transitions(
	mat::SparseMatrixCSC{Float64,Int64}
)
```

  * use default matrix Q_default

```
get_all_state_transitions()
```

## `get_canonical`

Return a dictionary of the Matrix `mat` in canonical form. If     - $n$ is total number of states in a given transition matrix,     - $t$ is the number of transient states, and     - $a$ is the number of absorping states,

```
then the orignal matrix (if it is an absorping discrete-time Markov Chain)
can be reduced to canonical form,
dictionary includes the following elements:

- `:R`: upper-right submatrix of ``A`` (``t \times a``)
- `:A`: input transition matrix `mat` in canonical form (``n \times n``)
- `:inds`: indices in ``A`` of original states in `mat`
- `:Q`: the transition matrix of transient states to transient states
    (dimension ``a \times a``)
    - **NOTE** this is not the original input matrix
- `:states_absorption`: all states in the input matrix that are absorbing
- `:states_transient`: all states in the input matrix that are transient
```

### Constructs

```
get_canonical(
    mat::SparseMatrixCSC{Float64,Int64} = transition_matrix_from_data_frame()
)
```

  * Note: `transition_matrix_from_data_frame` returns `Q_default` with no arguments

## `get_fundamental_matrix`

Returns the fundamental matrix of $Q$; requires `dict_canon` as input, which     is generated using `get_canonical`. The fundamental matrix is

```
``F = (I - Q)^{-1}``
```

### Constructs

```
get_fundamental_matrix(
	dict_canon::Dict
)
```

## `get_probability_of_absorption_state`

Returns the probability of absorption in each absorbing state $j$ given     starting state $i$. Requires `dict_canon` as input, which is generated     using `get_canonical`. The probability of the absorption state is calculated     directly as

```
``FR``

where ``F`` is the fundamental matrix (see `get_fundamental_matrix`) and ``R``
is the ``t 	imes a`` upper-right submatrix of the canonical-form of the
matrix `mat` used to generate `dict_canon`.
```

### Constructs

```
get_probability_of_absorption_state(
	dict_canon::Dict
)
```

## `get_state_attribute`

Retrieve an attribute associated with a state

### Constructs

```
get_state_attribute(
	state::Int64,
    attribute::String
)
```

## `get_state_entropies`

Get entropies associated with each state based on an input matrix `matrix`

### Constructs

```
get_state_entropies(
	matrix::SparseMatrixCSC{Float64, Int64}
)
```

```
get_state_entropies()
```

  * **Note** no argument will use the `MarkovModel` default matrix Q_default

## `find_absorption`

Return the $t \times a$ matrix $B_{ij}$ that gives the probability that a     walk starting in transient state $i$ (`1 \leq j \leq a``, where``t``is     the number transient states) will end up in state``j``(``1 \leq j \leq a``, where``a`` is the number of absorping states)

### Constructs

```
find_absorption(
	extraction_transition_states::Vector{Int64} = Vector{Int64}(),
	dict_variable_parameters::Dict{String, Float64} = configuration.dict_parameters_variable,
	configuration::Configuration = configuration,
	df_modspec::DataFrame = model_specification,
	extract_probabilites::Array{Tuple{Int64, Int64}, 1} = Array{Tuple{Int64, Int64}, 1}()
)
```

## `simulate_absorptions`

Simulate absorbptions over a DataFrame

### Constructs

```
simulate_absorptions(
	df_runs::DataFrame,
	dict_cols_to_params::Dict{Symbol, String},
	extraction_transition_states::Array{Int64,1} = Array{Int64,1}(),
	extract_transition_matrix_indices::Array{Tuple{Int64, Int64}, 1} = Array{Tuple{Int64, Int64}, 1}()
)
```

## `swap_inds!`

Swap indices in sparse array indices over an arbitrary number of dimensions     (vec*dim can be 1, 2, 3... etc vecs). This function supports `get*canonical`.

### Constructs

```
swap_inds!(
	ind_1::Int64,
	ind_2::Int64,
	vecs_dim::Vector{Int64}...
)
```

## `swap_states!`

Swap states in sparse array indices over an arbitrary number of dimensions     (vec*dim can be 1, 2, 3... etc vecs). This function supports `get*canonical`.

### Constructs

```
swap_states!(
	state_1::Int64,
	state_2::Int64,
	vecs_dim::Vector{Int64}...
)
```

## `transition_matrix_from_data_frame!`

Read a data frame and convert it to a transition matrix. The data frame must     have the following columns:     - $i$: row index in the matrix     - $j$: columns index in the matrix     - $expression$: the expression in the matrix

### Constructs

```
transition_matrix_from_data_frame(
	dict_variables::Dict{String, Float64},
	df_specification::DataFrame = model_specification,
	transition_correction_threshold::Float64 = configuration.transition_correction_threshold,
	stochastic_orientation::Symbol = configuration.transition_stochastic_orientation,
	return_orientation::Symbol = :row,
	check_states::Bool = true
)
```

```
transition_matrix_from_data_frame(
	df_specification::DataFrame = model_specification,
	configuration::Configuration = configuration,
	return_orientation::Symbol = :row,
	check_states::Bool = true
)
```

## `do_walk`

Do a random walk on a transition matrix `mat_transition` (default is     `Q_default`) starting from state `state_0`, and walk for `num_steps`.

```
Set `save_trials` to `true` to return the random trials used to conduct the
random walk.
```

### Constructs

```
do_walk(
	state_0::Int64,
	num_steps::Int64,
	save_trials::Bool = false,
	mat_transition::SparseMatrixCSC{Float64,Int64} = Q_default,
	vec_state_times::Vector{Int64} = attribute_states.get_ordered_attribute(state_time_attribute)
)
```

## `find_next_state`

Based on a vector of transitions (e.g., a row in a row-stochastic matrix) and a     random trial, return the next state.

### Constructs

using a random trial, find the next state

```
find_next_state(
	vec_transitions::SparseVector{Float64,Int64},
	trial::Float64
)
```

## `simulate_walks`

Run a simulation of multiple walks on a transition matrix.

### Constructs

```
simulate_walks(
    n_runs::Int64,
    n_time_periods::Int64,
    state_0::Int64;
    save_trials::Bool = false,
    return_type::Symbol = :DataFrame,
    mat_transition::SparseMatrixCSC{Float64,Int64} = Q_default,
    vec_state_times::Vector{Int64} = attribute_states.get_ordered_attribute(state_time_attribute),
    dict_entropy::Dict{Int64, Float64} = Dict{Int64, Float64}(),
    default_entropy::Float64 = 0.0
)
```

### Function Arguements

  * `n_runs`: number of walks to conduct
  * `n_time_periods`: maximum number of time periods to walk
  * `state_0`: starting state

### Keyword Arguements

  * `save_trials`: save off the random trials used to conduct the walk?
  * `mat_transition`: specify a different transition matrix. Default is   `Q_default`
  * `vec_state_times`: vector of times spent in each state (ordered by   `attribute_state`)
  * `dict_entropy`: include a dictionary of entropies by state to characterize   the entropy faced by a decisionmaker along the walk.
  * `default_entropy`: default entropy if no entropy is specified


###  MarkovModel includes methods that can be used for other matrices, but that default to the initialized matrix. 

The `transition_matrix_from_data_frame` method, when called without any arguments, builds the matrix generated by the input data frame `df_trans` from above.

In [6]:
# can be used with any data frame, but without a function argument, reads the default transition dataframe associated with the scenario
Q = mod_markov.transition_matrix_from_data_frame()


8×8 SparseMatrixCSC{Float64, Int64} with 18 stored entries:
  ⋅   0.3  0.7   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.3  0.5  0.2   ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   0.5  0.2   ⋅   0.1  0.2   ⋅ 
  ⋅    ⋅   0.4   ⋅    ⋅    ⋅   0.4  0.2
  ⋅    ⋅   0.5  0.4   ⋅    ⋅    ⋅   0.1
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

###  `MarkovModel` can be used to return the canonical form of the matrix (as well as the fundamental matrix)

Execute the next cell to return a dictionary giving canonical form A as well as applicable submatrices (Q/R/I) + states; empty arguments returns the default transition matrix. See `?MarkovModel` for more information on `get_canonical`


In [7]:
dict_canonical = mod_markov.get_canonical()
dict_canonical[:A]


8×8 SparseMatrixCSC{Float64, Int64} with 18 stored entries:
  ⋅   0.3   ⋅    ⋅    ⋅   0.7   ⋅    ⋅ 
  ⋅    ⋅   0.2   ⋅   0.5   ⋅   0.3   ⋅ 
  ⋅    ⋅    ⋅   0.4   ⋅   0.4   ⋅   0.2
  ⋅    ⋅    ⋅    ⋅    ⋅   0.5  0.4  0.1
  ⋅    ⋅   0.1  0.2   ⋅   0.5  0.2   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

###  Next, we can use the `MarkovModel` to perform a random walk based on the starting state. 

The default starting state is specified in the scenario attribute table and can be accessed as `dataset.attribute_scenario.field_maps["scenario_id_to_default_starting_state"]`.

Use `do_walk` to do a simple random walk in discrete time with two starting arguments: (`starting_state`, `n_time_steps`)

`do_walk` returns an ordered tuple with the following elements:

- discrete walk starting at state starting_state for n_time_stpes
- all state_changes
- the raw trials (if speciied -- this is an optional argument), 
- the set of unique transitions
- the time to absorption  

See `?MarkovModel` for more information on `do_walk`


In [8]:
state_0 = dataset.attribute_scenario.field_maps["scenario_id_to_default_starting_state"][scen]
print("default state_0 for scenario $(scen):\t$(state_0)")
walk_ex = mod_markov.do_walk(state_0, 1000);
walk_ex

default state_0 for scenario 4:	1

([1, 3, 3, 3, 3, 3, 3, 3, 3, 3  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Float64[], Set([(1, 3)]), 2)

###  To perform an ensemble of simulated walks, use `MarkovModel.simulate_walks()`

`simulate_walks` generally takes three arguments: (`n_runs`, `n_time_steps`, `starting_state`)


In the cell below, we run 10000 walks for 1000 periods using initial state `state_0` (defined above). The result of `MarkovModel.simulate_walks()` is a tuple of two data frames:
- `df_out`: a data frame long by `:run_id` (the index for simulation run) and `time_period` (the index for the number of time periods). The data frame includes two key output fields:
    - `state`: the state of the random walk at the given `time_period` for run `run_id`
    - `new_state`: a binary (0 or 1) indicating whether the state is a change from the last state (used for calculating metrics about state changes)
    
- `df_abs`: A summary data frame capturing information on absorption. For each `run_id`, it includes information on:
    - `absorption_time_period`: the time period that the walk reached an absorption state
    - `absorbed`: binary indicating whether or not the walk was absorbed
    - `n_state_changes`: the number of times the walk changed states.

####  Using the output data, information on entropies can be calculated.

See `?MarkovModel` for more information on `simulate_walks`. 
    - *Note: try running the cell below twice--Julia's JIT compiler can be slower the first time it is executed. Running it again should show noticeable speed improvements.*

In [10]:

t0 = time();

n_runs = 10000
n_time_periods = 1000
df_out, df_abs = mod_markov.simulate_walks(
    n_runs, 
    n_time_periods, 
    state_0; 
    save_trials = false, 
    return_type = :DataFrames
);

# format some information on entropies
df_entropies = DataFrame(
    dataset.field_state_key => Int64.(collect(1:size(mod_markov.Q_default)[1])), 
    :entropy => mod_markov.get_state_entropies()
)

df_entropies[:, dataset.field_scenario_key] = Int64.(ones(nrow(df_entropies))*scen)
t1 = time();

print("Total time for $(n_runs) runs of $(n_time_periods) steps: $(round(t1 - t0; digits = 2)) seconds.")

Total time for 10000 runs of 1000 steps: 2.22 seconds.

###  Next, internal tools allow the comparison of some simulated observations to analytical solutions
- Simulated soluations are generated using `MarkovModel.simulate_walks()`; times to absorption and the probabilities of ending in an absorption state given a starting state can be calculated from simulations (convergence probabilities0
- Analytical solutions are also available

See commenting and code in the cell below for a comparison of simulated convergence probabilities to those found in the analytical solution.

Note that the analytical solution is $B = FR$, where $F = (I - Q)^{-1}$ is the fundamental matrix and $R \in \mathbb{R}^{t \times a}$ is the upper-right submatrix of the canonical form, such that $R_{ij}$ gives the proabability of a transient state $i$ converging to absorption state $j$ (in this context, $i$ and $j$ are states in $A$, the canonical form of the transition matrix $Q$—not the original transition matrix $Q$). 

In [11]:

##  ANALYTIC SOLUTION FOR PROBABILITY OF ENDING IN EACH ABSORBING STATE (cols) BASED ON STARTING STATE (ROWS)

state_starting = 1
n_runs = 10000
n_time_periods = 2000


##  GET ANALYTIC PROBABILITY OF CONvERGENCE TO EACH ABSORPTION STATE GIVEN A STARTING STATE

dict_abs = mod_markov.find_absorption([state_starting])
dict_analytic = Dict(zip(dict_abs[:states_absorb], dict_abs[:B][1, :]))
df_analytic = DataFrame(dict_abs[:B], Symbol.(dict_abs[:states_absorb]))


##  SIMULATE WALKS AND CALCULATE STATISTICS FOR CONVERGENCE PROBABILITIES

df_sim = mod_markov.simulate_walks(n_runs, n_time_periods, state_starting);
# we ignore the small number of runs that don't converge by point t = 1000
df_sim_abs = filter(x -> ((x[:time_period] == n_time_periods) & (x[:state] in dict_abs[:states_absorb])), df_sim);
df_sim_abs[:, :count] .= 1
# combine and get mean
df_compare = combine(groupby(df_sim_abs[:, [:state, :count]], [:state]), :count => sum, renamecols = false)
df_compare[:, :simulated_probability_convergence] = df_compare[:, :count]/size(df_sim_abs)[1];
df_compare[:, :analytic_probability_convergence] = [dict_analytic[x] for x in df_compare[:, :state]];
rename!(df_compare, Dict(:state => :absorption_state))

,absorption_state,count,simulated_probability_convergence,analytic_probability_convergence
,Int64,Int64,Float64,Float64
1,3,8306,0.8306,0.835
2,4,1473,0.1473,0.144
3,8,221,0.0221,0.021


##  Finally, the `MarkovDecisionUtilities` package provides a single command can be used to execute simulations of all scenarios included in the configuration dataset

`MarkovDecisionUtilities.run_package_simulation!()` will run all scenarios from the configuration dataset using defaults from the configuration. These include:

- Default number of runs: `config.default_num_runs` 
- Default number of time steps: `config.default_num_time_steps`
- Default starting state: for each scenario, the default starting state comes from the scenario attribute table, which can be seen using `dataset.attribute_scenario.table`(applies to this Jupyter notebook, since `dataset` was initialized using the configuration data set specified in `config.dataset`)

- Running `run_package_simulation!()` will create a unique session key based off the date and time to track unique runs. Outputs are sent to a session key subdirectory created in `dir_out`. The directory contains analytical solutions, summary statistics, simulated walks, and all input data. These are stored in several files:
 
    - `run_package_simulation!()` writes `attribute_scenario.csv`, `attribute_state.cs`, `transition_matrices_by_scenario.csv`, and `complexity_modeling.config` to the session key subdirectory for archival purposes.
    - Analytical solutions by scenario (number `#`) are written to individual files called `analytical_solutions_$(dataset.field_scenario_key)_#.csv`. The files are long by starting state (field `starting_state`) and contain fields for the following info:
        1. the expectd time in each transient state $S$, given as `expected_time_in_state_$S$`;
        2. the expectd number of visits to state $S$, given as `expected_number_of_vists_to_state_$S$`; and
        2. the probability of absorption in state $S$, given as `probability_of_absorption_in_state_$S$`.
    - `expected_outcome_by_edge.csv`: The expected outcome (i.e., the simulation parameter outcome) across each edge (transitinon probability, $i \to j$) by scenario, calculated across all runs. 
    - `outcome_values_summary.csv`: summary outcome values calculated for each scenario (but indexed by scenario name). This table is useful for a quick, easily-readable comparison of scenarios.
    - `outcome_values.csv`: selected outcomes for each `scenario_id` and `run_id`, including: 
        - `state_period_TIMEPERIODMAX`: the state at the final time period (`TIMEPERIODMAX`);
        - `outcome_value`: the outcome value at final state or absorption;
        - `absorption_time_period`: the time period when the random walk is absorbed;
        - `absorbed`: whether or not the walk was absorbed;
        - `n_state_changes`: the number of state changes in the random walk;
        - `entropy_total_by_time`: total entropy over time;
        - `entropy_total_by_steps`: total entropy over all unique state changes;
        - `entropy_mean_by_time`: mean entropy over time;
        - `entropy_mean_by_steps`: mean entropy over all unique state changes;
    - `simulated_walks.csv`: random walks by scenario and run
    - `state_entropies.csv`: the input state entropies for each state and scenario.
    
For more information, see the docstring at ?run_package_simulation! (this can be tried in the cell immediately below before running the script in the following executable cell). 

In [12]:
?run_package_simulation!

search: run_package_simulation!



Run the simulation across all scenarios defined in the `MarkovDataset` dataset.

# Constructs

```
run_package_simulation!()
```

```
run_package_simulation!(
    dataset::MarkovDataset,
    n_runs::Int64,
    n_time_periods::Int64,
    dir_output::String = dir_out,
    # define the scenarios to run and the associated starting state
    dict_starting_states::Dict{Int64, Int64} = dataset.attribute_scenario.field_maps["scenario_id_to_default_starting_state"]
)
```

# Function Arguments

  * `dataset`: `MarkovDataset` to use to setup decision environment
  * `n_runs`: number of walks to simulate
  * `n_time_periods`: maximum number of time periods to run each walk for
  * `dir_output`: output directory to write subdirectory to
  * `dict_starting_states`: dictionary mapping each scenario to its starting state


In [13]:
# try running it
run_package_simulation!()
@info("run_pacakge_simulation! complete. See the output in $(dir_out)")

┌ Info: 
│ Initializting output using /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/ref/datasets/cuban_missile_crisis as /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/out/20221107_112843_output_cuban_missile_crisis...
└ @ MarkovDecisionUtilities /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/julia/MarkovDecisionUtilities.jl:291
┌ Info: 	Done.
└ @ MarkovDecisionUtilities /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/julia/MarkovDecisionUtilities.jl:293
┌ Info: 
│ Copying configuration file...
└ @ MarkovDecisionUtilities /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/julia/MarkovDecisionUtilities.jl:296
┌ Info: 	Done.
└ @ MarkovDecisionUtilities /Users/jsyme/Documents/Projects/FY21/PAF21/markov_decision_utilities/julia/MarkovDecisionUtilities.jl:298
┌ Info: 
│ Starting simulation of 1000 runs and 100 time periods...
│ 
└ @ MarkovDecisionUtilities /Users/jsyme/Documents/Project